# Model 3 Version 2.3.2.5
## Isolating trials of learning -- positive derivative value, negative derivative value
### Removing last 3 volumes using FSL ROI

In [23]:
%matplotlib inline
import shutil 
import os
from os.path import join, split, basename
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from glob import glob
from pylab import *

#cope01_FX_before_COND_incorr_run1_trl1_onset5.72.nii.gz
def onset_sort(x):
    x = x.split('_')[-1]
    return(x[5:-8])

subs = ['WMAZE_005']
'''
subs = ['WMAZE_002', 'WMAZE_004', 'WMAZE_005', 'WMAZE_006',
        'WMAZE_007', 'WMAZE_008', 'WMAZE_009', 'WMAZE_010', 'WMAZE_012',
        'WMAZE_017', 'WMAZE_018', 'WMAZE_019', 'WMAZE_020', 'WMAZE_021',
        'WMAZE_022', 'WMAZE_023', 'WMAZE_024', 'WMAZE_026', 'WMAZE_027']
'''

sets = ['set1', 'set2', 'set3']
runs = ['run1', 'run2', 'run3', 'run4', 'run5', 'run6']


for sub in subs:
    sub_dir = '/home/data/madlab/data/mri/wmaze/'
    
    #grab the derivative files created from the set-based learning analysis
    frst_deriv_files = glob(join(sub_dir, 
                                 'scanner_behav/{0}/Bprime_pmode_set*.txt'.format(sub))) 
    frst_deriv_files.sort() 
    
    #grab the learning curve file for B trials
    learning_files = glob(join(sub_dir, 'scanner_behav/{0}/B_pmode_set*.txt'.format(sub))) 
    learning_files.sort()
    
    #grab the 95% confidence file for B trials
    upper_95_files = glob(join(sub_dir, 
                               'scanner_behav/{0}/B_p95_set*.txt'.format(sub)))
    upper_95_files.sort()
    
    #grab all cope files created by LSS Model 5
    cope_files = glob(join(sub_dir, 
                          'frstlvl/wmaze_MRthesis/fixed_before_conditional/model3_2-3-2/{0}/'.format(sub),
                          'modelfit/contrasts/_estimate_model*/cope*_FX_before_COND_*corr_run*_trl*.nii.gz'))     
    cope_files.sort()
    
    #grab the behavioral files for all runs
    behav_runs = glob(join(sub_dir, 
                           'scanner_behav/{0}/{0}_wmazebl_2015*.txt'.format(sub))) 
    behav_runs.sort()
    
    #os.makedirs(join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                     #'fixed_before_conditional/model3_2-3-2-5_test/{0}/learning/'.format(sub)))
    
    #os.makedirs(join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                    # 'fixed_before_conditional/model3_2-3-2-5/{0}_test/nonlearning/'.format(sub)))
    
    
    #### LOADING AND ORGANIZING THE COPE FILES ####
    all_runs = []
    for curr_run in runs:
        #selects only the cope files containing the current run's number
        curr_run_files = np.array([f for f in cope_files if curr_run in f])
        #gets the onset time out of the file names using function
        onset_nums = [float(onset_sort(f)) for f in curr_run_files]
        sorted_nums = np.argsort(onset_nums)
        #arranges the actual files according to onset time
        curr_run_files = curr_run_files[sorted_nums]
        all_runs.append(curr_run_files)
        
    
    for i, curr_set in enumerate(sets):
        #load derivative, learning, and p95 files
        deriv_file = np.loadtxt(frst_deriv_files[i])
        learning_curve = np.loadtxt(learning_files[i]) 
        upper_95 = np.loadtxt(upper_95_files[i])
        
        #### COPE FILES ####
        #merge the two runs into one array for the current stim set
        curr_set_copes = np.concatenate((all_runs[i*2], all_runs[i*2+1])) 
        
        
        #### GETTING THE DERIV FILES TO MATCH NUMBER OF COPES ####
        #load behavioral files
        behav_run1 = pd.read_table(behav_runs[i*2])
        behav_run2 = pd.read_table(behav_runs[i*2+1])
        
        #info concerning onset time
        behav_os = np.empty(320, dtype=object)
        behav_os[:160] = behav_run1['StimOnset'].values
        behav_os[160:] = behav_run2['StimOnset'].values
        
        #info concerning subject response
        behav_resp = np.empty(320, dtype=object)
        behav_resp[:160] = behav_run1['Resp'].values
        behav_resp[160:] = behav_run2['Resp'].values
        
        #info concerning trial type
        behav_type = np.empty(320, dtype=object)
        behav_type[:160] = behav_run1['TrialType'].values
        behav_type[160:] = behav_run2['TrialType'].values
        
        trial_shift = behav_type[:-1] 
        trial_shift = np.insert(trial_shift, 0, -1)
        
        #indices of all Bs in the original dataset without preceeding BL trials
        b_indices = np.where((behav_type == 'B'))[0]
        #grabs B trials with preceeding BLs
        b_BL_indices = np.where((behav_type == 'B') & (trial_shift == 'BL'))[0]       
        
        #isolate bad Bs for removal in learning curve/derivative/p95 files
        bad_Bs = [] 
        bad_Bs.extend(b_BL_indices)
        for curr_B in b_indices:
            if not curr_B in bad_Bs:
                #identify in B trials which are non-response
                if behav_resp[curr_B] == 'NR': 
                    bad_Bs.append(curr_B)
                #indices if B trial comes first (observed on 001 run 6)    
                if curr_B in [0, 157, 158, 159, 160, 317, 318, 319]: 
                    bad_Bs.append(curr_B)
        bad_Bs.sort() 
        #print bad_Bs       
        
        #get the indices for the bad Bs within the group of Bs
        bad_B_ind = [j for j, f in enumerate(b_indices) if f in bad_Bs] 
        bad_B_ind.sort()
        #reverse order of Bs to be removed
        bad_B_ind = bad_B_ind[::-1]
        #print len(bad_B_ind)
                      
        #LEARNING CURVE FILES
        #create a temp version of learning_curve
        temp2 = list(learning_curve)
        #pop out the bad Bs starting from the end
        for curr_bad_B in bad_B_ind:
            temp2.pop(curr_bad_B)
        #save without the removed Bs    
        new_learning = np.array(temp2)
        #print len(new_learning)
        
        #P95 FILES
        temp3 = list(upper_95)
        for curr_bad_B in bad_B_ind:
            temp3.pop(curr_bad_B)  
        # new upper 95% without bad Bs    
        new_upper_95 = np.array(temp3)
        #print len(new_upper_95)
                
        
        #DERIV FILES
        temp = list(deriv_file)
        for curr_bad_B in bad_B_ind:
            temp.pop(curr_bad_B)   
        new_deriv = np.array(temp[:-1])
        #print new_deriv
        #print len(deriv_file)
        learning = np.where(new_deriv > 0)[0]
        nonlearning = np.where(new_deriv <= 0)[0]
        #print "learning"
        #print learning
        #print "nonlearning"
        #print nonlearning
         
        #remove the bad Bs from the B-list
        temp4 = list(b_indices)
        for curr_bad_B in bad_B_ind:
            temp4.pop(curr_bad_B)
        good_Bs = np.array(temp4)
        print "GB", len(good_Bs), good_Bs
        

        #convert original behavioral file indices to new B-specific index
        new_indices_B = []
        for n, curr_new_B in enumerate(good_Bs):
            new_indices_B.append(n)
        new_indices_B = np.array(new_indices_B)
        #print "New Bs"
        #print new_indices_B        
        #print new_indices_B
        #grabs the B trials with a positive value derivative
        b_learning = new_indices_B[learning]
        #print b_learning
        #print len(curr_set_copes)
        #grabs the B trials with a zero or negative value derivative
        b_nonlearning = new_indices_B[nonlearning]
        #print b_nonlearning
        fixed_learning_files = curr_set_copes[b_learning]
        fixed_nonlearning_files = curr_set_copes[b_nonlearning] 
        
        
'''        
        #copy and save selected learning files to new folder for merge script
        for curr_learning in fixed_learning_files:
            learning_basename =  os.path.basename(curr_learning)  
            shutil.copy2(curr_learning,
                         join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                              'fixed_before_conditional/model3_2-3-2-5/{0}/'.format(sub),
                              'learning/{0}'.format(learning_basename)))

        #copy and save selected after files to new folder for merge script   
        for curr_nonlearning in fixed_nonlearning_files:
            nonlearning_basename =  os.path.basename(curr_nonlearning)
            shutil.copy2(curr_nonlearning,
                         join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                              'fixed_before_conditional/model3_2-3-2-5/{0}/'.format(sub),
                              'nonlearning/{0}'.format(nonlearning_basename)))
'''        

GB 63 [  6   8  10  15  20  23  30  35  39  45  48  50  58  71  74  76  79  93
  96 101 103 109 112 115 119 124 128 130 136 145 147 153 156 168 176 185
 188 192 197 205 207 211 213 218 222 229 231 235 237 252 256 258 260 263
 268 274 278 282 300 307 309 312 316]
GB 61 [  1   6   8  11  15  19  29  32  35  39  45  51  62  69  76  78  81  86
  99 102 104 113 116 123 128 132 138 143 148 150 154 162 164 179 185 191
 194 196 200 207 213 218 222 225 229 231 237 240 242 247 254 258 268 272
 274 278 282 285 290 294 309]
GB 58 [  1   5  17  30  34  37  40  43  51  58  60  70  73  75  78  88  90  95
  98 101 105 113 117 120 124 137 140 147 151 154 156 162 165 168 171 175
 178 181 205 209 216 219 225 229 231 243 246 264 270 274 283 286 288 292
 296 298 309 312]


"        \n        #copy and save selected learning files to new folder for merge script\n        for curr_learning in fixed_learning_files:\n            learning_basename =  os.path.basename(curr_learning)  \n            shutil.copy2(curr_learning,\n                         join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',\n                              'fixed_before_conditional/model3_2-3-2-5/{0}/'.format(sub),\n                              'learning/{0}'.format(learning_basename)))\n\n        #copy and save selected after files to new folder for merge script   \n        for curr_nonlearning in fixed_nonlearning_files:\n            nonlearning_basename =  os.path.basename(curr_nonlearning)\n            shutil.copy2(curr_nonlearning,\n                         join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',\n                              'fixed_before_conditional/model3_2-3-2-5/{0}/'.format(sub),\n                              'nonlearning/{0}'.format(n